In [49]:
install.packages("quantmod")
library(quantmod)
install.packages("timeSeries")
library(timeSeries)
install.packages("rugarch")
library(rugarch)
install.packages("priceR")
library(priceR)


The downloaded binary packages are in
	/var/folders/rh/pfd6hrw52s35skb7d44l36nr0000gn/T//RtmpD9Sopu/downloaded_packages

The downloaded binary packages are in
	/var/folders/rh/pfd6hrw52s35skb7d44l36nr0000gn/T//RtmpD9Sopu/downloaded_packages

The downloaded binary packages are in
	/var/folders/rh/pfd6hrw52s35skb7d44l36nr0000gn/T//RtmpD9Sopu/downloaded_packages

The downloaded binary packages are in
	/var/folders/rh/pfd6hrw52s35skb7d44l36nr0000gn/T//RtmpD9Sopu/downloaded_packages


In [50]:
cur <- historical_exchange_rates("USD", to = "EUR",
                          start_date = "2010-01-01", end_date = "2020-06-30")

In [51]:
head(cur)

,date,one_USD_equivalent_to_x_EUR
,<date>,<dbl>
1,2010-01-01,0.694927
2,2010-01-02,0.694927
3,2010-01-03,0.694927
4,2010-01-04,0.693289
5,2010-01-05,0.696088
6,2010-01-06,0.694252


In [52]:
dates <- as.Date(as.character(cur[,1]), format="%Y-%m-%d")
head(dates)

[1] "2010-01-01" "2010-01-02" "2010-01-03" "2010-01-04" "2010-01-05"
[6] "2010-01-06"

In [53]:
# log returns
returns <- diff(log(cur[,2]))
head(returns)

[1]  0.000000000  0.000000000 -0.002359864  0.004029149 -0.002641082
[6]  0.005988498

In [54]:
# we will use 500 observations (window) to fit the ARMA and GARCH models
# we shift the window one step and fit the models again
window.length <- 100
forecasts.length <- length(returns) - window.length
forecasts <- vector(mode="numeric", length=forecasts.length)
# we need this feature: +1 for positive return -1 for negative return
directions <- vector(mode="numeric", length=forecasts.length)

In [58]:
# consider every trading day and calculate optimal model parameters
# make a prediction for next day's return
for (i in 0:forecasts.length) {
    # let's create rolling window
    roll.returns <- returns[(1+i):(window.length + i)]
    final.aic <- Inf
    final.order <- c(0,0,0)
    
    # ARMA model order p, q < 4 !!!
    for (p in 1:4) for (q in 1:4) {
        
      # because of the error=function(err) FALSE in case of an error, it returns  FALSE
      model <- tryCatch( arima(roll.returns, order = c(p,0,q)), error =  function( err ) FALSE, warning = function( err ) err)  
        
      # FALSE means the model fit was unsuccessful (needs more iteration for example)
      # we find the best model based in AIC !!!
      if (!is.logical(model)) {
          current.aic <- AIC(model)
          if (current.aic < final.aic){
              final.aic <- current.aic
              final.order <- c(p,0,q)
              final.arima <- arima(roll.returns, order =  final.order)
          }                  
       }
    }                  
                        

}


ERROR: Error in UseMethod("logLik"): no applicable method for 'logLik' applied to an object of class "c('simpleWarning', 'warning', 'condition')"


In [ ]:
    # We use GARCH(1,1) model
    # variance: GARCH is needed - mean: ARMA is needed
    # the errors have skew-generalized error distribution - sged
    spec = ugarchspec(variance.model <- list(garchOrder=c(1,1)),
                      mean.model <- list(armaOrder <- c(final.order[1], final.order[3]), include.mean = T),
                      distribution.model = "sqed")                  
                      
    # this is how we actually fit the GARCH(1,1) model
    # we just want to fit the model to the rolling window returns + hybrid tried difference
    fit = tryCatch(ugarchfit(spec, roll.returns, solver = 'hybrid'), error = function(e) e, warning = function(w) w)
     
    # let's make predictions with the fitted model
    # model does not always converge - assign value of 0 to prediction
    if (is(fit, "warning")) {
        forecasts[i+1] <- 0
        }   
    else {
        # we predict the return tomorrow (so 1 day ahead)
        next.day.forecast = ugarchforecast(fit, n.ahead = 1)
        # x variable contains the value of the return
        x =  next.day.forecast@forecast$seriesFor
        # we need the predictions: +1 (positive return) or -1 (negative return) / x[1] is
        directions[i+1] <- ifelse(x[1] > 0, 1, -1)
        # actual value of forecast: we predict returns
        forecasts[i+1] <- x[1]
    }